In [139]:
!pip install pathway panel bokeh --quiet

In [140]:
import numpy as np
import pandas as pd
import pathway as pw
from math import radians, sin, cos, sqrt, atan2
from bokeh.plotting import figure, output_notebook, show
output_notebook()

# Data Preprocessing

In [141]:
Data_url = 'https://drive.google.com/uc?id=1oZmW4LBq5JlVnWxWc7GdDgoxxQtXHL7o'

In [142]:
df = pd.read_csv(Data_url)

In [143]:
df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


In [144]:
df['timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], dayfirst=True)

In [145]:
# Encode categorical fields
df['TrafficEncoded'] = df['TrafficConditionNearby'].map({'low': 1, 'average': 2, 'high': 3})
vehicle_weights = {'car': 1.0, 'bike': 0.7, 'truck': 1.3, 'cycle': 0.5}
df['VehicleWeight'] = df['VehicleType'].map(vehicle_weights)

In [146]:
df['OccupancyRatio'] = df['Occupancy'] / df['Capacity']
df['IsSpecialDay'] = df['IsSpecialDay'].astype(int)

In [147]:
df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,timestamp,TrafficEncoded,VehicleWeight,OccupancyRatio
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00,1,1.0,0.105719
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00,2016-10-04 08:25:00,1,1.0,0.110919
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00,2016-10-04 08:59:00,1,1.0,0.138648
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00,2016-10-04 09:32:00,1,1.0,0.185442
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00,2016-10-04 09:59:00,1,0.7,0.259965


Model 1: Baseline Pricing


In [148]:
def baseline_price(df, alpha=5):
    df = df.copy()
    df['Price_Baseline'] = 10 + alpha * df['OccupancyRatio']
    return df

df = baseline_price(df)
df[['timestamp', 'SystemCodeNumber', 'OccupancyRatio', 'Price_Baseline']].head()

,timestamp,SystemCodeNumber,OccupancyRatio,Price_Baseline
0,2016-10-04 07:59:00,BHMBCCMKT01,0.105719,10.528596
1,2016-10-04 08:25:00,BHMBCCMKT01,0.110919,10.554593
2,2016-10-04 08:59:00,BHMBCCMKT01,0.138648,10.693241
3,2016-10-04 09:32:00,BHMBCCMKT01,0.185442,10.927210
4,2016-10-04 09:59:00,BHMBCCMKT01,0.259965,11.299827


Model 2: Demand-Based Pricing

In [149]:
def demand_price(df, λ=0.5, base_price=10):
    df = df.copy()
    df['Demand'] = (
        1.5 * df['OccupancyRatio'] +
        0.8 * df['QueueLength'] -
        0.7 * df['TrafficEncoded'] +
        1.2 * df['IsSpecialDay'] +
        0.6 * df['VehicleWeight']
    )
    # Normalize and price
    d_min, d_max = df['Demand'].min(), df['Demand'].max()
    df['DemandNorm'] = (df['Demand'] - d_min) / (d_max - d_min + 1e-8)
    df['Price_Demand'] = base_price * (1 + λ * df['DemandNorm'])
    df['Price_Demand'] = df['Price_Demand'].clip(lower=base_price * 0.5, upper=base_price * 2)
    return df

df = demand_price(df)

# Visualization

In [150]:
def plot_prices(lot_id):
    subset = df[df['SystemCodeNumber'] == lot_id]
    p = figure(title=f"Pricing Trends for {lot_id}", x_axis_type='datetime', height=300)
    p.line(subset['timestamp'], subset['Price_Baseline'], color='blue', legend_label='Baseline')
    p.line(subset['timestamp'], subset['Price_Demand'], color='green', legend_label='Demand-Based')
    p.legend.location = "top_left"
    show(p)

# Example plot
plot_prices("BHMBCCMKT01")

Real-Time Simulation Framework

In [163]:
import pathway as pw

BASE_PRICE = 10

@pw.udf
def compute_price(occ, queue, traffic, special, weight):
    occ_ratio = occ / 577
    demand = (1.5 * occ_ratio + 0.8 * queue - 0.7 * traffic + 1.2 * special + 0.6 * weight)
    norm = (demand - 2) / 6
    raw_price = BASE_PRICE * (1 + 0.5 * norm)
    return round(min(max(raw_price, 0.5 * BASE_PRICE), 2 * BASE_PRICE), 2)

source = pw.debug.table_from_markdown("""
Occupancy | QueueLength | TrafficEncoded | IsSpecialDay | VehicleWeight
100       | 2           | 1              | 0            | 1.0
300       | 5           | 3              | 1            | 1.3
500       | 4           | 2              | 0            | 1.2
""")

price_stream = source.select(
    Occupancy=source.Occupancy,
    QueueLength=source.QueueLength,
    TrafficEncoded=source.TrafficEncoded,
    IsSpecialDay=source.IsSpecialDay,
    VehicleWeight=source.VehicleWeight,
    price=compute_price(
        source.Occupancy,
        source.QueueLength,
        source.TrafficEncoded,
        source.IsSpecialDay,
        source.VehicleWeight
    )
)

# Use pw.io.fs.write correctly:
pw.io.fs.write(price_stream, "output_prices.jsonl", "json")

pw.run()


Output()

In [166]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.io import push_notebook
import time

output_notebook()

# Load the price output
df = pd.read_json("output_prices.jsonl", lines=True)

# Create a basic line plot (simulate time using row index)
p = figure(title="Dynamic Parking Price", x_axis_label='Time (Index)', y_axis_label='Price ($)')
p.line(df.index, df['price'], line_width=2, color="blue", legend_label="Parking Lot A")

show(p)